In [23]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="5" color="red">ch09_파일 입출력(i/o) 프로그래밍</font></b>

1. *.txt\
 김회원, 33, 아무동9\
 이회원, 22, 무시기동8

2. pickle파일(*.data) : 객체리스트 저장, 머신러닝모델 저장용, 메모장으로 열 수 없음

3. *.csv\
   name,age,address\
   "김회원",33,"아무동"\
   "이회원",22,"무시기동8"\
   단, 항목 사이에 스페이스 없어야함, ""은 스트링, 없을땐 숫자로 인식
   
4. *.json\
   실시간의 데이터를 구하고 싶을 때, 특정 API를 사용하게 되는데 그때 json파일로 많이 받는다.\
   [{'name':'김회원', 'age':33, 'address':'아무동9'}, \
    {'name':'이회원', 'age':22, 'address':'무시기동8'}]
    
5. *.h5 이나 *.hdf5  (=> 딥러닝 모델 저장)\
   HDVview로 읽을 수 있음

# 1절. txt 파일에 데이터 저장하고 불러오기

In [ ]:
# 지난시간 예외처리 방법
try:
    f = open('data/ch09.txt', 'w')
    # 데이터 출력
except FileNotFoundError as e:
    print(e)
finally:
    f.close()


In [ ]:
#수업을 위해 간단히 
f = open('data/ch09.txt', 'w')
print('쓰기가능한지 여부 :', f.writable())

In [ ]:
f.write('Hello\nWorld')
f.close()  # close() 쓰지않으면 저장되지 않음

In [ ]:
#close 조차 사용하지 않게 with 절 사용
with open('data/ch09.txt', 'w') as f:
    print('쓰기가능한지 여부 :', f.writable())
    f.write('Hello\nPython')

In [ ]:
# mode 
    # 'r' or 'rt' : text읽기 모드
    # 'rb' : 바이너리 읽기 모드
    # 'w' or 'wt' : text쓰기 모드 (파일이 있으면 덮어쓰기, 없으면 생성 후 쓰기)
    # 'wb' : 바이너리 쓰기 모드
    # 'a' or 'at': text 추가(append) 모드 (파일이 있으면 추가, 없으면 생성 후 쓰기)
    # 읽기모드에서는 '파일'이 없으면 예외/쓰기모드에서는 '폴더'가 없으면 예외
# 한글을 쓸때는 encoding
    # euc-kr (한글완성형) 믜x
    # cp949 (확장된 한글완성형) : open() 함수 기본 encoding방식 (win)
    # utf-8 (한글조합형) : open() 함수 기본 encoding방식 (mac, linux), 주피터노트북 기본
with open('data/ch09.txt', 'w') as f:
    print('쓰기가능한지 여부 :', f.writable())
    f.write('Hello\nPython')

In [ ]:
with open('data/ch09.txt', 'w', encoding='utf-8') as f:
    # 1방법
    f.write('홍길동, 33, 아무동9\n')
    f.write('김길동, 35, 무개동9\n')

    # 2방법
    textlist = ['홍길동, 33, 아무동9\n', '김길동, 35, 무개동9\n']
    for line in textlist:
        f.write(line)

    # 3방법
    f.writelines(textlist)

In [ ]:
# 한줄씩 읽기 
with open('data/ch09.txt', 'r', encoding='cp949') as f:
    line = f.readline()
    while line!='':
        print(line, end='')
        line = f.readline()

In [ ]:
# 모든 줄을 읽기
with open('data/ch09.txt', 'r') as f:
    lines = f.readlines()
    print(lines)

In [ ]:
# 모두 읽기
with open('data/ch09.txt', 'r') as f:
    lines = f.read()
    print(lines)

# 2절. 피클 파일 저장/불러오기
- 객체 혹은 객체리스트로 저장. 즉 클래스를 사용

## 2.1 형식이 있는 txt데이터 불러오기

In [ ]:
with open('data/ch09_member.txt', 'r') as f:
    line = f.readline()
    print(line)

In [ ]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                       '성년' if self.age >18 else '미성년',
                                       self.email,
                                       self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
                }    
    ## 객체간 같은 내용이면 True 값을 내기위해 아래 내용으로 인스턴스 함수 추가
    def __eq__(self, other):
        return self.name == other.name and\
            self.age == other.age and \
            self.email==other.email and\
            self.address==other.address

In [ ]:
user1 = Member('홍',20,'a@a.com','신림동')
user2 = Member('홍',20,'a@a.com','신림동')
print(user1==user2)    #  객체간 == 은 __eq__ 를 내부적으로 살펴본다.
print(user1.__eq__(user2))   # 같은 내용이면 True로 하고 싶다

In [ ]:
# 파일을 읽어서 한줄씩 읽고
# 형식이 있는 txt 파일 내용을 member list(피클 저장), 딕셔너리 list(데이터프레임) 저장
user_list = []  # 객체 리스트로 구현
user_dict = []  # 딕셔너리 리스트로 구현

with open('data/ch09_member.txt', 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()
# print(lines)

for line in lines:
    data = line.split(',')
    #print(data)
    name = data[0]
    age = int(data[1].strip()) # strip() : 좌우 공백(spac, \t, \n) 제거 
    email = data[2].strip()
    address = data[3].strip()
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())
    # user_dict.append(user.__dict__)
              

In [ ]:
print(user_list)

In [ ]:
for user in user_list:
    print(user)

In [ ]:
for user in user_dict:
    print(user)

## 2.2 피클링
- 객체 리스트(user_list) -> 피클 파일로 쓰기
- 피클 파일을 읽기 -> 객체 리스트(load_user_list)
- 저장 : pickle.dump(객체명, 스트림명)
- 로드 : 로드할객체명 = pickle.load(스트림명)

In [ ]:
# 피클링을 이용한 객체를 저장하기, 바이너리 모드로 써야한다
import pickle
with open('data/ch09_member.data', 'wb') as f:
    pickle.dump(user_list, f)

In [ ]:
# 피클링을 이용한 파일에서 객체 데이터로 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)

for data in load_user_list:
    print(data)

In [ ]:
load_user_list==user_list

In [ ]:
# 입력한 객체리스트와 로드한 객체리스트의 각 항목 값이 같은지 출력하고 확인하는 구문
for idx, (user, load_user) in enumerate(zip(user_list, load_user_list)):
    print(idx, user)
    print(idx, load_user)
    print(user==load_user)

In [ ]:
# 방대한 양을 일일이 확인할 수는 없다. 

result = []
for user, load_user in zip(user_list, load_user_list):
    result.append(user==load_user)
print(len(set(result))==1)
all(result)     # all() : 모두의 값이 같은지 체크하는 함수

# 3절. csv파일 읽기/쓰기
- 헤더가 없는 경우 : 헤더를 만들어줘야한다
- csv파일 <-> 리스트 데이터    3.1 읽기 / 3.2 쓰기
- csv파일 <-> 딕셔너리 리스트   3.3 읽기 / 3.4 쓰기

## 3.1 reader - csv파일 리스트로 읽기 

In [ ]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)  
    # csv.변수명(스트림명) : 한줄씩 읽어들여 리스트로 만드는 것을 반복하는 함수
    result = list(reader)   # with절이 끝나면 날아가므로 결과를 저장할 필요가 있어서 result에 저장
print(result)

In [ ]:
# ""(따옴표)가 없는 데이터는 numeric으로 csv.QUOTE_NONNUMERIC <단 실수로 나옴
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)  
    result = list(reader)
print(result)

In [ ]:
# 이 리스트, 딕셔너리로 바꿔봅시다
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)  
    result = list(reader)
dict_list = []
for data in result:
    dict_list.append({
        'name':data[0],
        'age':int(data[1]),
        'email':data[2],
        'address':data[3]
    })
# 길다. 하기 싫다. 간편하게 처리하는건 3.3에서 배울 예정.
# 질문. 해당 데이터에 헤더가 있었다면, __dict__ 쓸수 있었던건가?

print(dict_list)

## 3.2 writer

In [ ]:
user_list = [['홍길동', 20, 'kildong@hong.com', '서울시 관악구'],
            ['김길동', 40, 'kildong@kim.com', '서울시 강서']]

In [ ]:
with open('data/ch09_member1_write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(user_list)

In [ ]:
with open('data/ch09_member1_write.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)  # ""있는 건 있는걸로 ""없는 건 없는걸로 write
    writer.writerows(user_list)

## 3.3 DictReader
- csv파일을 딕셔너리로 읽음

In [ ]:
# 리스트는 csv.reader()를 썼지만 딕셔너리는 csv.DictReader() 클래스를 이용한다
with open('data/ch09_member3.csv', 'r', encoding='utf8') as f:
    dict_reader = csv.DictReader(f)
    for row in dict_reader:
        print(row)

# job 결측치가 있음. 부재시 출력하지 않겠다면?
with open('data/ch09_member3.csv', 'r', encoding='utf8') as f:
    dict_reader = csv.DictReader(f)
    dict_list = list(dict_reader)
for row in dict_reader:
    if row['job'] ==None:
        print()


In [ ]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames= ['Name', 'Age','Email','Address','Job']) # Job을 넣지 않으면 헤더 없이 
    print(list(dict_reader), end='\n')  
    # 전체출력. 여기에서 dict_reader는 파일에서 불러오는 참조타겟이다. 전체 출력시 처음부터 다 돌고 나면 빈곳을 가르키게된다
    for row in dict_reader:    # 위에서 참조가 끝났으므로 데이터를 가져오지 못한다.
        print(row)

In [ ]:
# 위에도 설명했지만 csv.DictReader로 생성한 리스트를 dict_reader에 할당했지만, 이는 엄연히 임시적인 참조값이라, 날아간다
# 따라서 새로운 변수를 생성해 결과값을 할당해주면 날아가지 않는다.
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames= ['Name', 'Age','Email','Address','Job']) # Job을 넣지 않으면 헤더 없이 
    result = list(dict_reader)  
    print(result)
    print()
    for row in result:    
        print(row)

In [ ]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames= ['Name', 'Age','Email','Address'], # Job을 넣지 않으면 헤더와 칼럼자리는 있으니 빈것으로 나옴
                                 restkey='Job')  # restkey='Job'  로 넣으면
    result = list(dict_reader)

for row in result:    
    print(row)

In [ ]:

for row in result:    
    print(row)

## 3.4 DictWriter
- 딕셔너리리스트 ->csv파일

In [ ]:
user1 = {'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구'}
user2 = {'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구'}
user3 = {'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구'}
user_list = [user1, user2, user3]
field_names = list(user1.keys())
field_names
user_list

In [ ]:
with open('data/ch09_member4.csv', 'w', newline="", encoding='utf-8') as f: # newline= 은 기본이 행바꿈
    dict_writer = csv.DictWriter(f,
                                fieldnames=field_names)
    dict_writer.writeheader()  # header 쓰기  / 위 fieldnames=field_names 에서 설정된 값을 넣는다.
    dict_writer.writerows(user_list)

## csv <-> 데이터프레임

In [ ]:
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv')  # 기본이 encoding='utf-8'
member

In [ ]:
type(member)

# 4절. JSON 데이터 저장하고 불러오고(dump, load)
- 딕셔너리 리스트 <-> JSON파일 (기본)
- 객체리스트 <-> JSON파일
## 4.1 dump(파일 출력)

In [ ]:
data = [{'name': '홍길동', 'age': 20, 'email': 'h@h.com', 'address': '서울시 관악구'},
         {'name': '신길동', 'age': 40, 'email': 's@h.com', 'address': '서울시 영등포구'},
         {'name': '김길동', 'age': 30, 'email': 'k@h.com', 'address': '서울시 동작구'}]

In [ ]:
# ensure_ascii 매개변수
    # True  : 비 ASCII 문자는 유니코드 형태로 저장
    # False  : 비 ASCII 문자 원래 형태로 저장
import json
with open('data/ch09_member.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(data,  # 딕셔너리 리스트가 기본
             jsonfile,  # 스트림 이름
             ensure_ascii=False,   # 비 아스키 코드 형태 유지
             indent='\t'  # 들여쓰기를 '  ' 만큼 할거야, 각 항목에 대한 줄바꿈이 포함되어 있다.
             )
    

In [ ]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                       self.age,
                                       self.email,
                                       self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
                }    
    
    def __eq__(self, other):
        if isinstance(other, Member):
            return self.__str__() == other.__str__()
#            return self.__dict__() == other.__dict__()
        else:
            return False

In [ ]:
user1 = Member('홍길동', 22, 'a@a.com', '신림동')
user2 = Member('홍길동', 22, 'a@a.com', '신림동')
print(user1.__eq__('홍길동'))
print(user1==user2)

In [ ]:
user_list = []
user_list.append(user1)
for row in user_list:
    print(row)

In [ ]:
for row in user1:
    print(row)

In [ ]:
member_list = [Member('홍길동', 22, 'a@a.com', '신림동'),
               Member('김길동', 22, 'a@a.com', '신림동'),
               Member('신길동', 22, 'a@a.com', '신림동')]


In [ ]:
import json
with open('data/ch09_member1.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(member_list,
             jsonfile,
             ensure_ascii=False,
             indent='\t',
             default=Member.as_dict #객체를 딕셔너리로 변경하는 함수가 필요
             )

## 4.2 load(파일입력)
- json파일 -> 딕셔너리 리스트 (기본)
- json파일 -> 객체리스트

In [ ]:
def as_member(dic):
    '매개변수로 딕셔너리를 받아 Member 객체를 return'
    return Member(dic.get('name'), dic.get('age'), dic.get('email'), dic.get('address'))

In [ ]:
member = as_member({'name': '홍길동', 'age': 20, 'email': 'h@h.com', 'address': '서울시 관악구'})
print(member)

In [ ]:
print(as_member)

In [ ]:
# 딕녀서너리로 로드
with open('data/ch09_member1.json', 'r', encoding='utf-8') as js:
    load_data = json.load(js)
load_data

In [ ]:
# json 파일을 딕셔너리로 불러서 객체로 변환하여 로드
with open('data/ch09_member1.json', 'r', encoding='utf-8') as js:
    load_data = json.load(js,
                         object_hook=as_member) # object_hook= 딕셔너리를 받아 Member객체로 리턴
    for data in load_data:
        print(data)

### Json -> 데이터프레임

In [ ]:
import pandas as pd
pd.read_json('data/ch09_member1.json')
# pd.함수() : encoding='utf-8' 이 기본값

In [ ]:
pd.read_json('data/한국토지주택공사_남양주왕숙2 토지이용계획도_20210924.json')

# 5절. HDF5 파일 읽기/쓰기
- h5py 사용
## 5.1 hdf5 파일 쓰기
- 딕셔너리 리스트를 hdf5파일에 쓰기
- 데이터프레임을 hdf5파일에 쓰기

In [2]:
# seaborn 패키지 : 시각화 패키지. 학습을 위한 데이터셋 다수
import seaborn as sns    # sns로 거의 공식화되어있다
iris_df = sns.load_dataset("iris") # 정말 많이 쓰는 함수
display(iris_df[::50])    # 데이터프레임은 display 해야함

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica


In [4]:
# hdf5 파일로 저장하기 : 데이터셋.to_hdf(저장할경로, key='데이터셋')
iris_df.to_hdf('data/ch09_iris.hdf5', key='iris')

## 5.2 hdf5 파일 읽기
- hdf5파일을 딕셔너리 리스트로 읽기
- hdf5파일을 데이터프레임으로 읽기

In [5]:
# 데이터프레임으로 읽기
import pandas as pd
load_iris_df = pd.read_hdf('data/ch09_iris.hdf5',key='iris')
load_iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
# 작성한 파일의 내용과 로드한 내용이 일치하는지 확인
iris_df == load_iris_df # 두 데이터 프레임 변수의 '각' 행열 값의 일치하는지를 반환
all(iris_df == load_iris_df)  # 두 데이터 프레임 변수의 '모든' 행열 값이 같은지를 반환

In [ ]:
#데이터프레임 변수를 딕셔너리 리스트로 변환 - ㅇㅇ.to_dict()
iris_dic= iris_df.to_dict() 
type(iris_dic)

In [ ]:
iris_dic.keys()  # 딕셔너리의 열 이름, 혹은 컬럼 네임

In [ ]:
# 

for group, value in iris_dic.items():
    print('그룹명 :', group)
    for key, data, in value.items():
        print('{}:{}'.format(str(key), data), end='\t')
    print()

In [ ]:
# 딕셔너리 리스트를 hdf파일로 쓰기
import h5py
with h5py.File('data/ch09_iris_dic.hdf5', 'w') as f:
    for group, value in iris_dic.items():
        grp = f.create_group(group)
        for key, data, in value.items():
            grp.create_dataset(str(key), data=data)

In [ ]:
with h5py.File('data/ch09_iris_dic.hdf5', 'r') as f:
    for group, value in f.items():
        print('그룹 :', group)
        for key, data in value.items():
            print(f'{key}:{data[()]}', end='\t')
            # data[()] : h5 DataSet타입을 가져올때 

In [ ]:
p={}
p['name'] = '홍'
p.update({'name':'홍'})
print(p)

In [ ]:
load_iris_dic = {}
load_iris_dic['s_l'] = {0:5.4, 1:4}
load_iris_dic['s_w'] = {0:5.4, 1:4}
load_iris_dic

In [ ]:
with h5py.File('data/ch09_iris_dic.hdf5', 'r') as f:
    for group, value in f.items():
        temp={}
        for key, data in value.items():
            temp[int(key)] = data[()]
        load_iris_dic[group] = temp   
            

In [ ]:
load_iris_dic

# 6절. 연습문제

## 실습형

### 클래스 만들기

In [1]:
class Customer:
    ' 고객데이터와 as_dic(), to_list_style(txt백업시), __str__()'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
        
    def as_dict(self): #객체를 딕셔너리데이터로 반환 (csv 파일 저장시)
        return {
            'name':self.name,
            'phone':self.phone,
            'email':self.email,
            'age':self.age,
            'grade':self.grade,
            'etc':self.etc
        }
    
    def to_list_style(self): #객체를 list return([ 홍길동 010 8999 9999, e@e.com, 20, 3, 까칠해]_
        temp = [
            self.name,
            self.phone,
            self.email,
            str(self.age),
            str(self.grade),
            self.etc
        ]
        return ','.join(temp)
            
    def __str__(self): # ***홍길동 010 8999 9999 e@e.com 20 까칠해
                return "{:>5}\t{:^4}\t{:13}\t{:16}\t{:3}\t{:5}".format('*'*self.grade,
                                                                        self.name,
                                                                        self.phone,
                                                                        self.email,
                                                                        self.age,
                                                                        self.etc
                                                                         )

In [2]:
# txt 파일내용 한 줄을 row 매개변수로 받아 Customer 객체로 return
def to_customer(row):
    '''
    row = "홍길동 , 010 8999 9999, e@e.com, 20, 3, 까칠해"(txt파일 내용)을 매개변수로 받아 Customer 객체로 return
    '''
    data = row.split(',')
    name = data[0].strip()
    phone = data[1].strip()
    email = data[2].strip()
    age = int(data[3].strip())
    grade = int(data[4].strip())
    etc = data[5].strip()
    return Customer(name, phone, email, age, grade, etc)

### 0. 로드

In [3]:
# 0.실행하면 data/ch09_customers.txt 파일의 내용을 load(customer_list)
# data/ch09_customers.txt 이 존재하지 않으면
#빈 data/ch09_customers.txt 파일을 생성하고
#데이터는 customer_list=[]

def load_customers():
    customer_list=[]
    try:
        with open('data/ch09_customs.txt', 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                customer = to_customer(line)
                customer_list.append(customer)
        print('데이터가 로드 되었습니다.')
    except:
        with open('data/ch09_customs.txt', 'w', encoding='utf-8') as f:
            f.writelines(customer_list)
            print('고객 리스트 파일이 없어, 새로운 파일을 생성합니다')
    return customer_list 

In [4]:
# 0. 로딩 테스트
customer_list = load_customers()
for row in customer_list:
    print(row)

데이터가 로드 되었습니다.
 ****	홍길동 	010-9999-9999	hong@mega_it.com	 20	까칠해  
  ***	고길동 	010-2929-5959	go@naver.com    	 44	둘리이슈 
    *	 둘리 	010-2222-2222	2222@2222.com   	100000	블랙리스트


In [5]:
for customer in customer_list:
    print(customer)

 ****	홍길동 	010-9999-9999	hong@mega_it.com	 20	까칠해  
  ***	고길동 	010-2929-5959	go@naver.com    	 44	둘리이슈 
    *	 둘리 	010-2222-2222	2222@2222.com   	100000	블랙리스트


### 1. 입력

In [6]:
# 1.입력
def fn1_insert_customer_info():
    '''
    사용자로 부터 이름, 번호, 메일, 나이, 등급, etc를 입력받아 Customer형 객체 반환
    '''
    name = input('이름 : ')
    phone = input('전화번호 : ')
    email = input('이메일 : ')
    while True:
        try:
            age = int(input('나이 : '))
            break
#            return age     함수가 끝나버린다
        except:
            print('나이를 잘못 입력하셨습니다. 정수를 입력하세요')
    while True:
        try:
            grade = int(input('고객등급(1~5) : '))
            if 1<=grade<=5:
                break
#                return grade   함수가 끝나버린다
        except:
            print('고객등급을 잘못 입력하셨습니다. 1~5 사이의 수를 입력하세요')
    etc = input('기타정보 : ')
    new_data=Customer(name, phone, email, age, grade, etc)
    return new_data

In [116]:
# 1. 입력 테스트
c1 = fn1_insert_customer_info()

이름 : 박진성
전화번호 : 010-2222-1112
이메일 : sdf@vasd.com
나이 : 37
고객등급(1~5) : 5
기타정보 : 신규


In [7]:
# 1. 입력 테스트 - 결과값
customer_list.append(c1)


NameError: name 'c1' is not defined

### 2. 전체 출력

In [8]:
# 2.전체 출력
def fn2_print_customers(customer_list):
#     with open('data/ch09_customs.txt', 'r', encoding='utf-8') as f:
#         lines = f.readlines()
        print('='*80)
        print('\t\t고객  정보')
        print('-'*80)
#        print('Grade\t','이름\t', '전화\t\t', '메일\t\t\t', '나이\t', '기타')
        print("{:>5}\t{:^4}\t{:13}\t{:16}\t{:3}\t{:5}".format('Grade', '이름', '전화', '메일', '나이','기타'))
        print('='*80)
        for customer in customer_list:
            print(customer)
        print('='*80)


In [9]:
# 2. 전체조회 테스트
fn2_print_customers(customer_list)

		고객  정보
--------------------------------------------------------------------------------
Grade	 이름 	전화           	메일              	나이 	기타   
 ****	홍길동 	010-9999-9999	hong@mega_it.com	 20	까칠해  
  ***	고길동 	010-2929-5959	go@naver.com    	 44	둘리이슈 
    *	 둘리 	010-2222-2222	2222@2222.com   	100000	블랙리스트


### 3. 삭제

In [186]:
# 3.삭제 동명이인이 있다고 할지 동명이인이 없다고 할지 미리 정해놓고 구현
# 동명이인 없는 버전
def fn3_delete_customer(customer_list):
    delete_name = input('삭제할 고객 이름은? ')
    for customer in customer_list:        
        if customer.name == delete_name:
            customer_list.remove(customer)
            print(f'{delete_name} 고객님 데이터를 삭제했습니다.')
            break
    else:
        print(f'{delete_name} 고객님을 찾지 못했습니다.')
        
#    return modified_customer_list
#     with open('data/ch09_customs.txt', 'w', encoding='utf-8') as f:
#         f.writelines(modified_customer_list)
        
    

In [187]:
# 3. 테스트
fn3_delete_customer(customer_list)

삭제할 고객 이름은? 박진성
박진성 고객님 데이터를 삭제했습니다.


In [194]:
# 동명이인이 있으며, 몇번째를 삭제할 것인지

def fn3_delete_customer_same(customer_list):
    delete_name = input('삭제할 고객 이름은? ')
    delelte_idx = []
    for idx, customer in enumerate(customer_list):        
        if customer.name == delete_name:
            delelte_idx.append(idx)
    if delelte_idx:
        for idx in delelte_idx[::-1]:  #마지막꺼부터 지워야 인덱스에 변화가 없다
            del customer_list[idx]
#            customer_list.pop(delelte_idx[::-1])
    print(f'{delete_name}님 {len(delelte_idx)}명 삭제했습니다.')


In [192]:
# 3번 기능테스트 - 동명이인 있는 버전
customer_list = load_customers()
fn3_delete_customer_same(customer_list)

데이터가 로드 되었습니다.
삭제할 고객 이름은? 박진성
박진성님 0명 삭제했습니다.


In [184]:
#customer_list = load_customers()
# print(modified_customer_list[0])
print()
#fn3_delete_customer(modified_customer_list[0])
for row in customer_list:
    print(row)



 ****	홍길동 	010-9999-9999	hong@mega_it.com	 20	까칠해  
  ***	고길동 	010-2929-5959	go@naver.com    	 44	둘리이슈 
    *	 둘리 	010-2222-2222	2222@2222.com   	100000	블랙리스트


### 4. 이름찾기

In [10]:
# 4.이름찾기 동명이인이 있다고 할지 없다고 할지 정한대로 구현
def fn4_search_customer(customer_list):
    search_name = input('찾을 고객 이름은? ')
    for customer in customer_list:
        if search_name == customer.name:
            print('='*80)
            print('\t\t고객  정보')
            print('-'*80)
            print("{:>5}\t{:^4}\t{:13}\t{:16}\t{:3}\t{:5}".format('Grade', '이름', '전화', '메일', '나이','기타'))
            print('='*80)
            print(customer)
            print('='*80)
            break
    else:
        print(f'{search_name} 고객님을 찾지 못했습니다.')
        

In [31]:
# 4.이름찾기 동명이인 없는 버전. 기존 함수 구현
def fn4_search_customer(customer_list):
    search_name = input('찾을 고객 이름은? ')
    search_list = []
    for customer in customer_list:
        if search_name == customer.name:
            search_list.append(customer)
    if search_list:
        fn2_print_customers(search_list)
    else:
        print(f'{search_name} 고객님을 찾지 못했습니다.')
        

In [32]:
fn4_search_customer(customer_list)

찾을 고객 이름은? 둘리
		고객  정보
--------------------------------------------------------------------------------
Grade	 이름 	전화           	메일              	나이 	기타   
    *	 둘리 	010-2222-2222	2222@2222.com   	100000	블랙리스트


In [14]:
for row in customer_list:
    print(row)

 ****	홍길동 	010-9999-9999	hong@mega_it.com	 20	까칠해  
  ***	고길동 	010-2929-5959	go@naver.com    	 44	둘리이슈 
    *	 둘리 	010-2222-2222	2222@2222.com   	100000	블랙리스트


In [26]:
# 4.이름찾기 동명이인이 있는 것으로 구현
def fn4_search_customer_2(customer_list):
    search_name = input('찾을 고객 이름은? ')
    search_idx = []
    for idx, customer in enumerate(customer_list):
        if search_name == customer.name:
            search_idx.append(idx)
    
    if search_idx:
        print('='*80)
        print('\t\t고객  정보')
        print('-'*80)
        print("{:>5}\t{:^4}\t{:13}\t{:16}\t{:3}\t{:5}".format('Grade', '이름', '전화', '메일', '나이','기타'))
        print('='*80)
        for idx in search_idx:
            print(customer_list[idx])
        print('='*80)

    



In [28]:
customer_list = load_customers()
fn4_search_customer_2(customer_list)

데이터가 로드 되었습니다.
찾을 고객 이름은? 홍길동
		고객  정보
--------------------------------------------------------------------------------
Grade	 이름 	전화           	메일              	나이 	기타   
 ****	홍길동 	010-9999-9999	hong@mega_it.com	 20	까칠해  
 ****	홍길동 	010-3333-9999	go@naver.com_it.com	 20	들까칠해 


In [207]:
#customer_list = load_customers()
fn4_search_customer(customer_list)
list(search_idx)

찾을 고객 이름은? 홍길동
		고객  정보
--------------------------------------------------------------------------------
Grade	 이름 	전화           	메일              	나이 	기타   


TypeError: 'Customer' object is not subscriptable

In [204]:
for row in customer_list:
    print(row)

 ****	홍길동 	010-9999-9999	hong@mega_it.com	 20	까칠해  
  ***	고길동 	010-2929-5959	go@naver.com    	 44	둘리이슈 
    *	 둘리 	010-2222-2222	2222@2222.com   	100000	블랙리스트
